# Tutorial for HPS single DDX4 

In [ ]:
# load packages
import numpy as np
import pandas as pd
import sys
try:
    import openmm as mm
    import openmm.app as app
    import openmm.unit as unit
except ImportError:
    import simtk.openmm as mm
    import simtk.openmm.app as app
    import simtk.unit as unit

import mdtraj
try:
    import nglview
except ImportError:
    print('Please install nglview to visualize molecules in the jupyter notebooks.')

sys.path.append('../../')
from openabc.forcefields.parsers import HPSParser
from openabc.forcefields import HPSModel
from openabc.utils.helper_functions import build_straight_CA_chain, write_pdb

# set simulation platform
platform_name = 'CPU'

We simulate a DDX4 protein with HPS model. We begin with building a straight CA atom chain with DDX4 sequence. We use `HPSParser` to parse the CA model.

In [ ]:
sequence = 'MGDEDWEAEINPHMSSYVPIFEKDRYSGENGDNFNRTPASSSEMDDGPSRRDHFMKSGFASGRNFGNRDAGECNKRDNTSTMGGFGVGKSFGNRGFSNSRFEDGDSSGFWRESSNDCEDNPTRNRGFSKRGGYRDGNNSEASGPYRRGGRGSFRGCRGGFGLGSPNNDLDPDECMQRTGGLFGSRRPVLSGTGNGDTSQSRSGSGSERGGYKGLNEEVITGSGKNSWKSEAEGGES'
ca_pdb = 'init_DDX4_CA.pdb'
ca_atoms = build_straight_CA_chain(sequence, r0=0.38)
write_pdb(ca_atoms, ca_pdb)
protein_parser = HPSParser(ca_pdb)
protein = HPSModel()
protein.append_mol(protein_parser)

Use the Urry scale optimal parameter ($\mu=1$ and $\Delta=0.08$) and run the simulation. 

In [ ]:
top = app.PDBFile(ca_pdb).getTopology()
init_coord = app.PDBFile(ca_pdb).getPositions()
protein.create_system(top, box_a=100, box_b=100, box_c=100)
protein.add_protein_bonds(force_group=1)
protein.add_contacts('Urry', mu=1, delta=0.08, force_group=2)
protein.add_dh_elec(force_group=3)
temperature = 300*unit.kelvin
friction_coeff = 1/unit.picosecond
timestep = 10*unit.femtosecond
integrator = mm.LangevinMiddleIntegrator(temperature, friction_coeff, timestep)
platform_name = 'CPU'
protein.set_simulation(integrator, platform_name, init_coord=init_coord)
protein.simulation.minimizeEnergy()
output_interval = 100
output_dcd = 'output_single_DDX4.dcd'
protein.add_reporters(output_interval, output_dcd)
protein.simulation.context.setVelocitiesToTemperature(temperature)
protein.simulation.step(500)

In [ ]:
# visualize trajectory
traj = mdtraj.load_dcd('output_single_DDX4.dcd', top=ca_pdb)
traj.xyz -= np.mean(traj.xyz, axis=1, keepdims=True) # realign to the origin
view = nglview.show_mdtraj(traj)
view